<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#统计学习笔记（06）：决策树" data-toc-modified-id="统计学习笔记（06）：决策树-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>统计学习笔记（06）：决策树</a></span><ul class="toc-item"><li><span><a href="#特征选择：信息增益与信息增益比" data-toc-modified-id="特征选择：信息增益与信息增益比-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>特征选择：信息增益与信息增益比</a></span></li><li><span><a href="#ID3、C4.5-和-CART" data-toc-modified-id="ID3、C4.5-和-CART-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>ID3、C4.5 和 CART</a></span></li></ul></li></ul></div>

#### 统计学习笔记（06）：决策树
---
**2020-03-18**

分类决策树是一种描述对实例进行分类的树形结构。用决策树分类，从根节点开始，对实例的某一特征进行测试，根据测试结果将实例分配到其子节点。每一个子节点对应该特征的一个取值，这样递归地对实例进行测试和分配，直到达到叶节点，将实例分到叶节点的类中。 

可以将决策树看做是一个 if-then 规则的集合，决策树的路径对应的 if-then 规则集合互斥并且完备。  

决策树也表示给定特征条件下的条件概率分布，这一条件概率分布定义在特征空间的一个分布（partition）上，该特征空间划分为互不相交的单元（cell）或区域（region），并在每个单元定义一个类的概率分布。假设 $X$ 是表示特征的随机变量，$Y$ 是表示类的随机变量，那么这个条件概率分布可以表示为 $P(Y\vert X)$，$X$ 取值于给定划分下单元的集合，$Y$ 取值于类的集合。各叶节点（单元）上的条件概率往往偏向某一个类，即属于某一个类的概率较大，决策树分类时将该节点的实例强行分到条件概率大的那一类去。

决策树学习本质上是从训练数据集中归纳出一组分类规则，需要找到的是一个与训练数据矛盾较小的决策树，同时具有较好的泛化能力。决策树学习的策略中的损失函数通常是正则化的极大似然函数。  

决策树学习算法包含特征选择、决策树生成和决策树剪枝的过程。

##### 特征选择：信息增益与信息增益比

特征选择在于选取对训练数据具有分类能力的特征，通常的特征选择的准则是信息增益或信息增益比。

熵（entropy）是表示随机变量的不确定性的度量，设 $X$ 是一个取有限个值的随机变量，其概率分布为 $$P(X=x_i)=p_i, \;\;\;\; i = 1,2, \cdots, n$$ 
则随机变量 $X$ 的熵定义为 $$H(X) = - \sum_{i=1}^n p_i \log p_i$$ 其中的对数函数一般取以2或 $e$ 为底，单位是bit。熵只依赖于 $X$ de 分布，取值无关，所以可以记为
$$H(p) = - \sum_{i=1}^n p_i \log p_i$$  熵越大，随机变量的不确定性就越大。

设有随机变量 $(X, Y)$，其联合概率分布为 $$P(X=x_i, Y=y_j) = p_{ij}, \;\;\;\; i = 1,2,\cdots, n; \;\;\;\; j = 1,2,\cdots, m$$
条件熵 $H(Y \vert X)$ 表示在已知随机变量 $X$ 的条件下随机变量 $Y$ 的不确定性，随机变量 $X$ 给定条件下随机变量 $Y$ 的条件熵（conditional entropy） $H(Y \vert X)$，定义为 $X$ 给定条件下 $Y$ 的条件概率分布的熵对 $X$ 的数学期望
$$H(Y \vert X) = \sum_{i=1}^n p_i H(Y \vert X=x_i) $$ 
这里 $p_i = P(X=x_i), \;\;\;\; i = 1,2,\cdots, n$

信息增益（information gain）表示得知特征 $X$ 的信息而使得类 $Y$ 的信息的不确定性减少的程度。特征 $A$ 对训练数据集 $D$ 的信息增益 $g(D,A)$ 定义为集合 $D$ 的经验熵 $H(D)$ 与特征 $A$ 给定条件下的经验条件熵 $H(D\vert A)$ 之差
$$g(D,A) = H(D) - H(D\vert A)$$ 一般将熵与条件熵的差称为互信息（mutual information）。

设训练数据集为 $D$，$\vert D \vert$ 表示其样本的容量（个数）。设有 $K$ 个类 $C_k, k = 1,2, \cdots, K$， $\vert C_k \vert$ 为属于类 $C_k$ 的样本数。再设特征 $A$ 有 $n$ 个不同的取值 $\{ a_1, a_2, \cdots, a_n\}$，根据特征 $A$ 的取值将 $D$ 划分为 $n$ 个子集 $D_n$，$\vert D_i \vert$ 表示 $D_i$ 的样本个数。记子集 $D_i$ 中属于类 $C_k$ 的样本的集合为 $D_{ik}$， 即 $D_ik = D_i \cap C_k$。

信息增益的计算是 $$g(D,A) = H(D) - H(D\vert A)$$

其中 $$H(D) = -\sum_{k=1}^K \frac{\vert C_k \vert}{\vert D \vert} \log_2 \frac{\vert C_k \vert}{\vert D \vert}$$
$$H(D \vert A) = \sum_{i=1}^n \frac{\vert D_i \vert}{\vert D \vert} H(D_i) \sum_{k=1}^K \frac{\vert D_{ik} \vert}{\vert D_i \vert} \log_2 \frac{\vert D_{ik} \vert}{\vert D_i \vert}$$
这样就可以通过比较不同特征的信息增益值，来选择最优特征。

另一种特征选择的方式是用信息增益比： $$g_R(D,A) = \frac{g(D,A)}{H(D)}$$

##### ID3、C4.5 和 CART

传统分类决策树的算法主要有 **ID3**和**C4.5**，前者使用信息增益值来选择特征，后者使用信息增益比。